### 1. 

En el archivo "logistic_regression_df_class" hemos visto un ejemplo multiclase. Elimina del dataframe todas las filas que se correspondan con la clase valor "1".

Ahora, realiza el ejercicio con el nuevo dataframe:

- ¿Se mejora la precisión del algoritmo con dos clases? ¿por qué?

LogisticRegression() es una clase que tiene varios parámetros de entrada. Investiga (modifica, prueba) los argumentos y comenta si modificando algunas de ellas se mejora el porcentaje de acierto del problema (probar al menos 2 diferentes)

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html


In [21]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import confusion_matrix

Cargamos los datos en un dataframe y lo filtramos según los requisitos del ejercicio

In [22]:
df = pd.read_csv('../data/usuarios_win_mac_lin.csv')
df = df[~(df['clase'] == 1)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130 entries, 0 to 169
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   duracion  130 non-null    float64
 1   paginas   130 non-null    int64  
 2   acciones  130 non-null    int64  
 3   valor     130 non-null    int64  
 4   clase     130 non-null    int64  
dtypes: float64(1), int64(4)
memory usage: 6.1 KB


In [23]:
df.head()

,duracion,paginas,acciones,valor,clase
0,7.0,2,4,8,2
1,21.0,2,6,6,2
2,57.0,2,4,4,2
3,101.0,3,6,12,2
4,109.0,2,6,12,2


In [24]:
df['clase'].value_counts()
# Comprobamos que sólo hay dos clases

0    86
2    44
Name: clase, dtype: int64

Establecemos una semilla para todo el notebook

In [25]:
seed = 42

Creamos un modelo y una partición de los datos para tener un conjunto de entrenamiento y uno de test

In [26]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('clase', axis=1), df['clase'], test_size=0.2, random_state=seed)

In [27]:
model = LogisticRegression(max_iter=100)

Vamos a hacer una validación cruzada y ver los resultados

In [28]:
cv = KFold(n_splits=5, shuffle=True, random_state=seed)
scores = cross_val_score(estimator=model, X=X_train, y=y_train, scoring='accuracy', cv=cv, n_jobs=-1)

In [29]:
scores
# Los resultados son bastantes buenos

array([0.95238095, 0.95238095, 0.95238095, 1.        , 0.85      ])

Vamos a entrenar el modelo y ver los resultados

In [30]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
print(f'Score con el conjunto de test {model.score(X_test, y_test)}\nScore con el conjunto de entrenamiento {model.score(X_train, y_train)}')

Score con el conjunto de test 0.8461538461538461
Score con el conjunto de entrenamiento 0.9615384615384616


Vemos que los resultados son bastantes mejores que cuando teniamos tres clases

In [32]:
# Vamos a crear la matriz de confusión
pd.DataFrame(confusion_matrix(y_pred=y_test, y_true=model.predict(X_test)))

,0,1
0,13,0
1,4,9


In [33]:
# Entrenamos el modelo con todos los datos
model.fit(df.iloc[:, :-1], df['clase'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
print(f'Score con todo el conjunto de datos {model.score(df.iloc[:, :-1], df["clase"])}')

Score con todo el conjunto de datos 0.9538461538461539


Vamos a probar cambiando el parámetro 'tol' que representa el valor mínimo para parar las iteracciones. Este parámetro esta por defecto establecido en 0.0001

In [35]:
model = LogisticRegression(max_iter=100, tol=1e-10)
model.fit(X_train, y_train)
model.score(X_test, y_test)
# Vemos que el resultado no cambia

0.8461538461538461

Ahora probamos cambiando el parámetro 'C' que es la inversa de la intensidad de regularización. Cuanto más bajo, mas intenso será la regularización. Este parámetro esta por defecto establecido en 1

In [36]:
model = LogisticRegression(max_iter=100, C=0.5)
model.fit(X_train, y_train)
model.score(X_test, y_test)
# El resultado empeora

0.8461538461538461

Cambiamos el algoritmo de optimización a 'liblinear' y cambiamos el tipo de regulador a 'l1'

In [37]:
model = LogisticRegression(max_iter=100, penalty='l1', solver='liblinear')
model.fit(X_train, y_train)
model.score(X_test, y_test)
# El resultado no cambia

0.9230769230769231

Por último entrenamos el modelo con todos los datos

In [38]:
model.fit(df.drop('clase', axis=1), df['clase'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)